In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, normalize
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

from utils import mod_df, drop_features, inverse_mod_X, inverse_mod_y, apply_savgol_filter, apply_median_filter, apply_maximum_filter, apply_is_zero
from my_model import create_model_1, create_model_1_1, create_model_1_2, create_model_1_3, create_model_2, create_model_2_1
tf.__version__

In [ ]:
train_df = pd.read_csv(os.path.join('data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('data', 'unionTest.csv'))

In [ ]:
X_train, y_train = mod_df(train_df)
X_test, y_test = mod_df(test_df)

In [ ]:
from multiprocessing.pool import ThreadPool as Pool

def feature_engineering(df):
    result = df.copy()
    
    # drop features
    result = result.drop(['id', 'timestep'], axis=1)
    # result = drop_features(result, [15, 16, 17, 18, 20, 21, 23, 24])
    
    # add features
    FEATURE_COLUMNS = result.columns.to_list()
    for col in FEATURE_COLUMNS:
        feature = result[col]
        feature = feature.to_numpy()
        result[f'{col}_savgol'] = apply_savgol_filter(feature, window_size=11, polynomial=2)
        result[f'{col}_median'] = apply_median_filter(feature)
        # result[f'{col}_max'] = apply_maximum_filter(feature)
        result[f'{col}_sav_med'] = apply_median_filter(apply_savgol_filter(feature, window_size=21), window_size=5)
        result[f'{col}_is_zero'] = apply_is_zero(feature)
    
    # modify features
    FEATURE_COLUMNS = result.columns.to_list()
    for col in FEATURE_COLUMNS:
        feature = result[col]
        feature = feature.to_numpy()
        # result[col] = apply_savgol_filter(feature)
        # result[col] = apply_median_filter(feature)
        # result[col] = apply_maximum_filter(feature)
        # result[col] = apply_median_filter(apply_savgol_filter(feature, window_size=21), window_size=5)
        
     
    return result

# pool = Pool(4)
X_train_1 = feature_engineering(X_train)
X_test_1 = feature_engineering(X_test)

In [ ]:
# print(X_train_1['0X_savgol'][:5], X_train_1['0X'][:5])

In [ ]:
# tmp = X_train_1['0X'].value_counts()
# tmp2 = np.array([tmp.index.to_numpy(), tmp.to_numpy()]).T
# tmp2[0:100, :]

In [ ]:
def get_scaler(scaler_name='MinMaxScaler'):
    if scaler_name == 'RobustScaler':
        scaler = RobustScaler()
    elif scaler_name == 'MinMaxScaler':
        scaler = MinMaxScaler()
    return scaler
# scaler_name = 'RobustScaler'
scaler_name = 'MinMaxScaler'
scaler = get_scaler(scaler_name)

In [ ]:
X_train_scaled = scaler.fit_transform(X_train_1)
X_test_scaled = scaler.transform(X_test_1)

X_train_scaled.shape

In [ ]:
# joblib.dump(scaler, os.path.join('saved_scaler', f'{scaler_name}.pkl'))

In [ ]:
X_train_modified, y_train_modified = inverse_mod_X(X_train_scaled), inverse_mod_y(y_train)
X_test_modified, y_test_modified = inverse_mod_X(X_test_scaled), inverse_mod_y(y_test)
print(X_train_modified.shape, y_train_modified.shape)

In [ ]:
input_shape, n_output = (X_train_modified.shape[-2], X_train_modified.shape[-1]), y_train_modified.shape[1]
model = create_model_1(input_shape, n_output)
# model = create_model_1_0(input_shape, n_output)
# model = create_model_1_1(input_shape, n_output)
# model = create_model_1_2(input_shape, n_output)
# model = create_model_2(input_shape, n_output)
# model = create_model_2_1(input_shape, n_output)
# model = Sequential([
#         InputLayer(input_shape),
#         Bidirectional(LSTM(32, return_sequences=True)),
#         Bidirectional(LSTM(32, return_sequences=False)),
#         Dense(25, activation='selu'),
#         Dense(n_output, activation="softmax")
#     ])
model.summary()

In [ ]:
optimizer = Adam(learning_rate=1e-3)
model.compile(
                optimizer=optimizer,
                loss=CategoricalCrossentropy(),
                metrics=[
                    'accuracy'
                ]
              )

In [ ]:
reduce_lr = ReduceLROnPlateau(
                                monitor='val_loss',
                                factor=0.5,
                                patience=4,
                                min_lr=1e-4
                              )
early_stopping = EarlyStopping(
                                monitor='loss',
                                patience=6
                              )

In [ ]:
history = model.fit(X_train_modified, y_train_modified,
          batch_size=32,
          epochs=35,
          shuffle=True,
          validation_split=0.2,
          callbacks=[
              reduce_lr,
              early_stopping,
              ]
          )

In [ ]:
from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model.save(os.path.join("saved_models", f"{current_time}_{scaler_name}.h5"))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
train_predict = model.predict(X_train_modified)
train_predict = np.argmax(train_predict, axis=1)+1
train_real = np.argmax(y_train_modified, axis=1)+1

# for i in range(len(y_train_modified)):
#     print(f"Index:{i}, Predict:{train_predict[i]}, Real:{train_real[i]}")

In [ ]:
f1_train = f1_score(train_real, train_predict)
accuracy_train = accuracy_score(train_real, train_predict)
# print(f"f1: {f1_train:.4f}\naccuracy: {accuracy_train:.4f}")
print(classification_report(train_real, train_predict, digits=4))
print("---------------------------------------------------------")
sns.heatmap(confusion_matrix(train_real, train_predict),annot = True,fmt = '2.0f')
print()

In [ ]:
test_predict_0 = model.predict(X_test_modified)
test_predict = np.argmax(test_predict_0, axis=1)+1
test_real = np.argmax(y_test_modified, axis=1)+1

# for i in range(len(y_test)):
#     print(f"Index:{i}, Predict:{test_real[i]}, Real:{test_real[i]}")

In [ ]:
# for i in range(len(y_test)):
    # print(f"Index:{i}, Predict:{test_predict_0[i]}, Real:{test_real[i]}")
# plt.plot(test_predict==test_real)
# plt.plot(np.max(test_predict_0, axis=1))
# print(np.max(test_predict_0, axis=1))
# plt.plot(test_real==2)
# plt.plot(test_predict_0[:, 1]>0.35)
# for i in np.arange(0, 1, 0.05):
#     print(i, sum((test_predict_0[:, 1]>i)==(test_real==2)))
#     plt.plot((test_predict_0[:, 1]>i)==(test_real==2))
#     plt.show()

In [ ]:
f1_test = f1_score(test_real, test_predict)
accuracy_test = accuracy_score(test_real, test_predict)
# print(f"f1: {f1_test:.4f}\naccuracy: {accuracy_test:.4f}")
print(classification_report(test_real, test_predict, digits=4))
print("---------------------------------------------------------")
sns.heatmap(confusion_matrix(test_real, test_predict),annot = True,fmt = '2.0f')
print()